## Bike Utilization: 2017 

In [17]:
#Create list of 2017 month filenames
months_2017 = []

for x in range(12):
    if x+1<10:
        month = '2017' + str(0)+str(x+1)
    if x+1>=10:
        month = '2017' + str(x+1)
    months_2017.append(month)

In [18]:
df_2017 = pd.DataFrame()

for x in months_2017:
    
    filename = x+'-citibike-tripdata.csv'
    
    csv_path = os.path.join('data', filename)
    csv = pd.read_csv(csv_path)
    csv = pd.DataFrame(csv)
    
    #Convert columns names to lowercase and remove spaces to ensure uniform headers
    csv.columns = csv.columns.str.lower()
    csv.columns = csv.columns.str.replace(' ','')

    csv = csv[['startstationid','startstationname','startstationlatitude','startstationlongitude',
            'endstationid', 'endstationname','endstationlatitude','endstationlongitude',
            'bikeid']]
    
    df_2017 = df_2017.append(csv)
    #For tracking progress, print x, the filename month
    print(f'{x}')

201701
201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712


In [21]:
df_2017.head()

,startstationid,startstationname,startstationlatitude,startstationlongitude,endstationid,endstationname,endstationlatitude,endstationlongitude,bikeid
0,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542
1,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136
2,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,18147
3,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,21211
4,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,26819


In [19]:
#Cross check that this is the toal number of trips in 2017
len(df_2017)

16364657

In [48]:
bikeid_2017 = df_2017.groupby(['bikeid']).size().rename('trip_count').reset_index()
bikeid_2017 = bikeid_2017.sort_values(by=['trip_count'], ascending=False)
bikeid_2017.head()

,bikeid,trip_count
7067,25738,2514
6614,25275,2409
8374,27161,2376
7799,26565,2370
8324,27111,2349


In [49]:
bikeid_2017.to_csv('bikeid_2017.csv')

In [50]:
median_trip_count = bikeid_2017['trip_count'].median()
median_trip_count

1237.0

In [51]:
mean_trip_count = bikeid_2017['trip_count'].mean()
mean_trip_count

1152.1160940580119

## Master List of Stations

In [372]:
#Create Master List of all station id's and location

#Start stations
start_stat_2017 = df_2017[['startstationid','startstationname','startstationlatitude','startstationlongitude']]
start_stat_2017 = start_stat_2017.rename(columns = {'startstationid':'stat_id','startstationlatitude':'stat_lat',
                                                   'startstationlongitude':'stat_lon','startstationname':'stat_name'})
start_stat_2017_unique =  start_stat_2017.drop_duplicates('stat_id')
total_start_stat_2017 = len(start_stat_2017_unique)

#End stations
end_stat_2017 = df_2017[['endstationid','endstationname','endstationlatitude','endstationlongitude']]
end_stat_2017 = end_stat_2017.rename(columns = {'endstationid':'stat_id','endstationlatitude':'stat_lat',
                                                   'endstationlongitude':'stat_lon','endstationname':'stat_name'})
end_stat_2017_unique =  end_stat_2017.drop_duplicates('stat_id')
total_end_stat_2017 = len(end_stat_2017_unique)

#Merge Start and End Stations
station_index = pd.merge(start_stat_2017_unique, end_stat_2017_unique, how='outer', on='stat_id')
total_station_index = len(station_index)

station_index = station_index[['stat_id','stat_name_x','stat_lat_x','stat_lon_x']]
station_index = station_index.rename(columns = {'stat_name_x':'stat_name','stat_lat_x':'stat_lat','stat_lon_x':'stat_lon'})

station_index.to_csv("station_master_list.csv")

#Double check Merge went as expected
print(f"Start Stations: {total_start_stat_2017}; End Stations: {total_end_stat_2017}; Total in Index:{total_station_index}")

Start Stations: 811; End Stations: 847; Total in Index:848


## Most Popular Stations in Feb 2018

In [373]:
import os
import pandas as pd

filename = '201802_citibikenyc_tripdata.csv'

csv_path = os.path.join('data', filename)
csv = pd.read_csv(csv_path)
csv = pd.DataFrame(csv)

#Convert columns names to lowercase and remove spaces to ensure uniform headers
csv.columns = csv.columns.str.lower()
csv.columns = csv.columns.str.replace(' ','')

csv = csv[['startstationid','endstationid']]

df_201802 = csv

In [374]:
#count start station id
start_201802 = df_201802['startstationid'].value_counts().to_frame().reset_index()
start_201802.columns = ['stat_id','2018_start_count']

#count end station id
end_201802 = df_201802['endstationid'].value_counts().to_frame().reset_index()
end_201802.columns = ['stat_id','2018_end_count']

#merge the start and end counts
stat_count_201802 = pd.merge(start_201802,end_201802, on='stat_id')
stat_count_201802.head()

,stat_id,2018_start_count,2018_end_count
0,519,9059,9060
1,402,5729,6059
2,435,5721,5867
3,497,5362,5817
4,3255,5268,5332


In [375]:
#Merge with master station list
stat_count_201802 = pd.merge(stat_count_201802 ,station_index, on='stat_id')
stat_count_201802.head()

,stat_id,2018_start_count,2018_end_count,stat_name,stat_lat,stat_lon
0,519,9059,9060,Pershing Square North,40.751873,-73.977706
1,402,5729,6059,Broadway & E 22 St,40.740343,-73.989551
2,435,5721,5867,W 21 St & 6 Ave,40.741740,-73.994156
3,497,5362,5817,E 17 St & Broadway,40.737050,-73.990093
4,3255,5268,5332,8 Ave & W 31 St,40.750585,-73.994685


In [376]:
stat_count_201802.to_csv('stat_count_201802.csv')